In [796]:
using Pkg
Pkg.add("DataFrames")
using DataFrames
using LinearAlgebra

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`


### Implementação das substituições reversa e direta

In [797]:
function backward_subs(U, b)
    """
    U: matriz triangular superior nxn com elementos na diagonal diferentes de 0
    b: um vetor nx1
    """
    # pegando o tamanho n que precisamos
    n = size(b)[1]
    # para guardar o resultado
    x = zeros(n)
    # para iteração
    rows = eachrow(U)

    for i in reverse(eachindex(rows)) # for n, n-1...
        x[i] = b[i]
        for j in reverse(i+1:n) # realizando as subtrações de cada elemento que já conhecemos
            x[i] = x[i] - U[i, j]*x[j]
        end
        x[i] = x[i] / U[i, i] # realizando a divisão para encontrar x[i]
    end

    return x
end

backward_subs (generic function with 1 method)

In [798]:
# definindo o tamanho do sistema
n = 4
# gerando uma matriz aleatória de tamanho n
U = randn(n, n)
# tornando a matriz aleatória uma triangular superior
U = [i <= j ? U[i, j] : 0 for i in 1:n, j in 1:n]
# definindo um vetor aleatorio x para multiplicar U
x = randn(n)
b = U*x

x=backward_subs(U, b)
# verificando se o resultado está correto
# utilizando a função norm devido aos erros numéricos
norm(U*x - b)

1.1102230246251565e-16

In [799]:
function forward_subs(L, b)
    """
    L: matriz triangular inferior nxn com elementos na diagonal diferentes de 0
    b: um vetor nx1
    """
    # pegando o tamanho n que precisamos
    n = size(b)[1]
    # para guardar o resultado
    x = zeros(n)
    # para iteração
    for i in 1:n # for 1, 2...
        x[i] = b[i]
        for j in 1:i-1 # realizando as subtrações de cada elemento que já conhecemos
            x[i] = x[i] - L[i, j]*x[j]
        end
        x[i] = x[i] / L[i, i] # realizando a divisão para encontrar x[i]
    end

    return x
end

forward_subs (generic function with 1 method)

In [800]:
# definindo o tamanho do sistema
n = 5
# gerando uma matriz aleatória de tamanho n
L = randn(n, n)
# tornando a matriz aleatória uma triangular inferior
L = [i >= j ? L[i, j] : 0 for i in 1:n, j in 1:n]
# definindo um vetor aleatorio x para multiplicar L
x = randn(n)
b = L*x

x=forward_subs(L, b)
# verificando se o resultado está correto
# utilizando a função norm devido aos erros numéricos
norm(L*x - b)

2.220446049250313e-16

### Questão 1

Construindo a matriz de usuários: cada linha representa um usuário, cada coluna representa um gênero de filme (romance e ação, respectivamente).

In [801]:
users_labels = ["A", "B", "C", "D", "E"]
users = [0.9 0.1; 0.8 0.2; 0.3 0.7; 0.6 0.4; 0.0 1.0]

5×2 Matrix{Float64}:
 0.9  0.1
 0.8  0.2
 0.3  0.7
 0.6  0.4
 0.0  1.0

In [802]:
# usuário e seus gostos (o primeiro elemento é para romance, o segundo para ação)
for (user_label, preference) in zip(users_labels, eachrow(users))
    println("User $user_label: $preference")
end

User A: [0.9, 0.1]
User B: [0.8, 0.2]
User C: [0.3, 0.7]
User D: [0.6, 0.4]
User E: [0.0, 1.0]


Construindo a matriz de filmes: cada linha representa um filme, cada coluna representa um gênero (romance e ação, respectivamente).

In [803]:
movies_labels = ["Titanic", "Rocky", "The Hobbit", "Fight Club", "Jurassic Park"]
movies = [0.9 0.1; 0.1 0.9; 0.5 0.5; 0.0 1.0; 0.2 0.8]

5×2 Matrix{Float64}:
 0.9  0.1
 0.1  0.9
 0.5  0.5
 0.0  1.0
 0.2  0.8

In [804]:
# filmes e seu pertencimento nos gêneros
# cada elemento é um gênero de filme (romance e ação, respectivamente)
for (movie_label, genres) in zip(movies_labels, eachrow(movies))
    println("$movie_label: $genres")
end

Titanic: [0.9, 0.1]
Rocky: [0.1, 0.9]
The Hobbit: [0.5, 0.5]
Fight Club: [0.0, 1.0]
Jurassic Park: [0.2, 0.8]


Matematicamente, sabemos da impossibilidade de multiplicação das matrizes, já que temos uma $M_{users}$ $\in \mathbb{R}^{5\times2}$ e uma $M_{movies}$ $\in \mathbb{R}^{5\times2}$. Sabemos também que a multiplicação matricial irá operar sobre linha $\times$ coluna. 

Nossa matriz de usuários está construída de forma que as linhas sejam cada usuário e as colunas representem seus gostos em relação aos gêneros de filme. Se a matriz de filme estivesse construída de forma que as colunas representassem os filmes, intuitivamente faria sentido a relação usuárioXfilme, já que seria linha $\times$ coluna.

Podemos fazer $M_{users}M_{movies}^{T}$, o que representa a afinidade dos gêneros relacionados a cada usuário com os gêneros relacionados a cada filme, fazendo a relação usuárioXfilme.

In [805]:
# construindo a transposta da matriz de filmes
movies'

2×5 adjoint(::Matrix{Float64}) with eltype Float64:
 0.9  0.1  0.5  0.0  0.2
 0.1  0.9  0.5  1.0  0.8

Temos agora uma matriz de filmes que está em $\mathbb{R}^{2\times5}$. Cada linha representa um gênero (romance e ação, respectivamente) e cada coluna representa um filme. 

Sendo assim, a relação usuário $\times$ filme é expressa pelo produto linha $\times$ coluna, que relacionará os gostos de gênero de um usuário com os gêneros de um filme.

A seguir, uma exemplificação.

In [806]:
# uma matriz 1x2: uma linha (usuário A) e duas colunas (gostos para romance e ação)
user_a = [0.9 0.1]

1×2 Matrix{Float64}:
 0.9  0.1

In [807]:
# um vetor de 2x1: duas linhas (valores para a afinidade dos gêneros romance e ação no filme) e uma coluna (filme Titanic)
titanic = [0.9, 0.1]

2-element Vector{Float64}:
 0.9
 0.1

In [808]:
# produto entre matriz e vetor, que nos dá a informação sobre a possível afinidade do usuário A com o filme titanic, a partir dos dados anteriormente destacados
user_a * titanic

1-element Vector{Float64}:
 0.8200000000000001

Sabemos que a multiplicação matricial irá repetir esse processo linha $\times$ coluna. Podemos executar a operação e entender os dados:

In [809]:
M = users * movies'

5×5 Matrix{Float64}:
 0.82  0.18  0.5  0.1  0.26
 0.74  0.26  0.5  0.2  0.32
 0.34  0.66  0.5  0.7  0.62
 0.58  0.42  0.5  0.4  0.44
 0.1   0.9   0.5  1.0  0.8

Cada linha representa um usuário e cada coluna representa um filme. O valor $M[user][movie]$ da matriz informa a possível afinidade de um determinado usuário com um determinado filme.

In [810]:
# a "coluna" User é apenas para formatação
# suponha que A, B, ..., E são as linhas da matriz com um label para visualização

df = DataFrame(M, Symbol.(movies_labels))
df = insertcols!(df, 1, :User => users_labels)
df

Row,User,Titanic,Rocky,The Hobbit,Fight Club,Jurassic Park
,String,Float64,Float64,Float64,Float64,Float64
1,A,0.82,0.18,0.5,0.1,0.26
2,B,0.74,0.26,0.5,0.2,0.32
3,C,0.34,0.66,0.5,0.7,0.62
4,D,0.58,0.42,0.5,0.4,0.44
5,E,0.1,0.9,0.5,1.0,0.8


Cada coluna é uma "dimensão" no espaço de filmes existentes dessa matriz. Cada linha, que representa um usuário, tem então um ponto em cada "dimensão", em cada filme.
Se cada usuário é um ponto nesse espaço de filmes, se analisarmos, por exemplo, o comprimento do vetor $A-B$, teremos a noção do quão distante estão os usuários A e B; nesse caso, na nossa intepretação, o quão diferentes eles são em relação aos filmes existentes. Quanto menor esse vetor, mais próximos eles estão, geometricamente, e isso se traduz em uma maior similaridade de gostos.

Temos então:

In [811]:
similarity = [] # lista que guardará tuplas na forma (Usuário 1, Usuário 2, Distância entre eles)
M_rows = eachrow(M) # lista com as linhas de M
n = size(M_rows)[1] # para melhor leitura

for user_idx in 1:n-1 
    for other_user_idx in user_idx+1:n # iterando sobre todas as duplas possíveis de usuários
        v_1 = M[user_idx, :] # pegando o vetor que guarda os valores do primeiro usuário
        v_2 = M[other_user_idx, :] # pegando o vetor que guarda os valores do segundo usuário

        v_1_label = users_labels[user_idx] # para leitura 
        v_2_label = users_labels[other_user_idx] # para leitura

        # calculando o tamanho da distância entre eles
        dist = norm(v_1-v_2)
        # adicionando na lista
        push!(similarity, (v_1_label, v_2_label, dist))
    end
end

In [812]:
similarity # (usuário 1, usuário 2, distância entre eles)

10-element Vector{Any}:
 ("A", "B", 0.16248076809271922)
 ("A", "C", 0.9748846085563152)
 ("A", "D", 0.4874423042781577)
 ("A", "E", 1.462326912834473)
 ("B", "C", 0.8124038404635959)
 ("B", "D", 0.32496153618543844)
 ("B", "E", 1.2998461447417538)
 ("C", "D", 0.48744230427815755)
 ("C", "E", 0.4874423042781577)
 ("D", "E", 0.9748846085563153)

Quanto menor a distância, maior a similaridade entre os usuários em questão. Sendo assim, iremos ordená-los por afinidade de gostos:

In [813]:
sort!(similarity, by = x -> x[3]) # ordenando pelos valores numéricos (as distâncias)

10-element Vector{Any}:
 ("A", "B", 0.16248076809271922)
 ("B", "D", 0.32496153618543844)
 ("C", "D", 0.48744230427815755)
 ("A", "D", 0.4874423042781577)
 ("C", "E", 0.4874423042781577)
 ("B", "C", 0.8124038404635959)
 ("A", "C", 0.9748846085563152)
 ("D", "E", 0.9748846085563153)
 ("B", "E", 1.2998461447417538)
 ("A", "E", 1.462326912834473)

Podemos observar que os usuários A e B possuem gostos bem similares, sendo os mais parecidos, assim como há uma boa afinidade entre B e D. Os gostos entre os usuários A e E; B e E são os mais distintos nesses dados.

### Questão 2

No PDF.

### Questão 3

Consideremos:
- branco = 0
- preto = 1

A partir da imagem da bandeira (o azul agora é preto), podemos montar a matriz A da seguinte forma:

In [814]:
# considerei um tamanho arbitrário (mas verossímil) para as colunas 
A = [1 1 0 1 1 1 1 1 1 1 1 1 1 1 
     1 1 0 1 1 0 0 0 0 0 0 0 0 0
     0 0 0 0 0 1 1 1 1 1 1 1 1 1
     1 1 0 1 1 0 0 0 0 0 0 0 0 0
     1 1 0 1 1 1 1 1 1 1 1 1 1 1
     0 0 0 0 0 0 0 0 0 0 0 0 0 0
     1 1 1 1 1 1 1 1 1 1 1 1 1 1
     0 0 0 0 0 0 0 0 0 0 0 0 0 0
     1 1 1 1 1 1 1 1 1 1 1 1 1 1
    ]

9×14 Matrix{Int64}:
 1  1  0  1  1  1  1  1  1  1  1  1  1  1
 1  1  0  1  1  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  1  1  1  1  1  1  1  1  1
 1  1  0  1  1  0  0  0  0  0  0  0  0  0
 1  1  0  1  1  1  1  1  1  1  1  1  1  1
 0  0  0  0  0  0  0  0  0  0  0  0  0  0
 1  1  1  1  1  1  1  1  1  1  1  1  1  1
 0  0  0  0  0  0  0  0  0  0  0  0  0  0
 1  1  1  1  1  1  1  1  1  1  1  1  1  1

In [815]:
rank(A)

3

É do nosso interesse decompor essa matriz $A$ em um produto $A=BC^t$.

A partir da imagem da própria bandeira, é imediata a percepção de que podemos usar a coluna $1$ (melhor visualizada na matriz numérica) para construir as colunas $2$, $4$ e $5$, que são iguais à coluna $1$; ou seja, $a_2 =  a_4 = a_5 = 1a_1$. Assim, a primeira coluna da nossa matriz $B$ é $a_1$.

A segunda percepção é de que não há nenhuma forma de combinar a coluna $3$, que representa a parte que inclui a linha vertical, para formar alguma outra coluna da bandeira, nem de combinar outras colunas de forma a formar $a_3$. Dessa forma, a incluimos em nossa matriz $B$.

Além dessas duas, podemos notar apenas mais um padrão que ainda não foi expresso, que são as colunas que, quando juntas, formam a parte puramente horizontal da bandeira, a começar por $a_6$ (primeira parte da bandeira onde há apenas linhas horizontais alternadas entre azul e branco). A partir de $a_5$, podemos reconstruir toda essa parte.

Então nossa $B$ pode ser escrita. Para $C$, basta sabermos "o quanto precisamos" da combinação de $B$ para multiplicar e voltar a $A$:

In [816]:
B = [1 0 1
     1 0 0
     0 0 1
     1 0 0
     1 0 1
     0 0 0
     1 1 1
     0 0 0
     1 1 1] 

9×3 Matrix{Int64}:
 1  0  1
 1  0  0
 0  0  1
 1  0  0
 1  0  1
 0  0  0
 1  1  1
 0  0  0
 1  1  1

Para multiplicarmos e voltarmos a $A$, teremos que ter uma matriz $3\times14$, que é elementar de achar: basta pensarmos o quanto precisamos de cada linha de $B$ para juntar com cada coluna da matriz requerida que nos faça retornar à $A$, o que é possível fazer "no olho", sem a necessidade de um algorítmo prático.

In [817]:
# pensamos na coluna como "o quanto precisamos dos elementos das linhas" de B para retornar a a
# para a primeira linha de A, precisamos que as colunas sejam a_1, a_1 (primeira coluna de B), a_3 (segunda coluna de B), a_1, a_1, e a_5 (terceira coluna de B)
# essa "montagem" da primeira linha pode ser vista pelas colunas de C
# "precisamos de 1B_1, 1B_1, 1B_2, 1B_1, 1B_1 e 1B_3 até o final da bandeira
C = [1 1 0 1 1 0 0 0 0 0 0 0 0 0
     0 0 1 0 0 0 0 0 0 0 0 0 0 0
     0 0 0 0 0 1 1 1 1 1 1 1 1 1
    ]

3×14 Matrix{Int64}:
 1  1  0  1  1  0  0  0  0  0  0  0  0  0
 0  0  1  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  1  1  1  1  1  1  1  1  1

As colunas da matriz $B$ são as peças fundamentais para que seja possível criar a bandeira. A partir de repetições delas (combinações lineares "multiplicando por $1$"), podemos construir a imagem da bandeira, o que é codificado pela matriz C, que contém o quanto dessas repetições precisamos fazer para cada uma das peças fundamentais. 

Essas "peças fundamentais" são independentes, ou seja, não é possível criar uma a partir de outra, por isso elas definem o posto da matriz.

#### Países que possuem bandeira com posto = 1
Polônia e Indonésia

Considerando:
- branco = 0
- vermelho = 1

In [818]:
A_poland = [
    0 0 0 0 0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0 0 0 0 0
    1 1 1 1 1 1 1 1 1 1 1 1 1 1
    1 1 1 1 1 1 1 1 1 1 1 1 1 1
    1 1 1 1 1 1 1 1 1 1 1 1 1 1
]

6×14 Matrix{Int64}:
 0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0
 1  1  1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1  1

É elementar perceber que temos uma repetição da primeira coluna em todas as outras. Em outras palavras, precisamos apenas da coluna $a_1$ para conseguir construir todo o resto da bandeira:

In [819]:
B_poland = [
    0
    0
    0
    1
    1
    1
]

6-element Vector{Int64}:
 0
 0
 0
 1
 1
 1

E repetimos uma vez essa coluna em toda a bandeira, sendo assim, C é formado apenas por $1$'s.

In [820]:
C_poland = [1 1 1 1 1 1 1 1 1 1 1 1 1 1]

1×14 Matrix{Int64}:
 1  1  1  1  1  1  1  1  1  1  1  1  1  1

Analogamente, para a Indonésia:

In [821]:
A_indonesia = [
    1 1 1 1 1 1 1 1 1 1 1 1 1 1
    1 1 1 1 1 1 1 1 1 1 1 1 1 1
    1 1 1 1 1 1 1 1 1 1 1 1 1 1
    0 0 0 0 0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0 0 0 0 0
]

# a primeira coluna é suficiente para construir todas as outras
B_indonesia = [
    1
    1
    1
    0
    0
    0
]

# os elementos de B são todos repetidos apenas uma vez
C_indonesia = [1 1 1 1 1 1 1 1 1 1 1 1 1 1]

1×14 Matrix{Int64}:
 1  1  1  1  1  1  1  1  1  1  1  1  1  1

#### Países que possuem bandeira com posto = 2
Inglaterra e Finlândia

Considerando:
- branco = 0
- vermelho = 1

In [822]:
A_england = [
    0 0 0 0 0 0 0 1 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 1 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 1 0 0 0 0 0 0 0
    1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
    0 0 0 0 0 0 0 1 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 1 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 1 0 0 0 0 0 0 0
]

7×15 Matrix{Int64}:
 0  0  0  0  0  0  0  1  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  1  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  1  0  0  0  0  0  0  0
 1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
 0  0  0  0  0  0  0  1  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  1  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  1  0  0  0  0  0  0  0

Podemos notar que precisamos de duas colunas dessa vez: a primeira, $a_1$, que se repete por quase toda a bandeira, e a coluna $a_8$, que é única e não é possível obtê-la a partir de combinações das outras.
Sendo assim:

In [823]:
B_england = [
    0 1
    0 1
    0 1
    1 1 
    0 1
    0 1 
    0 1
]

7×2 Matrix{Int64}:
 0  1
 0  1
 0  1
 1  1
 0  1
 0  1
 0  1

É elementar também notar que a bandeira é formada apenas por repetições da primeira coluna, com a exceção da coluna $a_8$, logo:

In [824]:
C_england = [
    1 1 1 1 1 1 1 0 1 1 1 1 1 1 1
    0 0 0 0 0 0 0 1 0 0 0 0 0 0 0
]

2×15 Matrix{Int64}:
 1  1  1  1  1  1  1  0  1  1  1  1  1  1  1
 0  0  0  0  0  0  0  1  0  0  0  0  0  0  0

Analogamente, para a Finlândia:

In [825]:
A_finland = [
    0 0 0 0 0 0 0 2 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 2 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 2 0 0 0 0 0 0 0
    2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
    0 0 0 0 0 0 0 2 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 2 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 2 0 0 0 0 0 0 0
]

# as colunas a_1 e a_8 são suficientes para reconstruir toda a bandeira
B_finland = [
    0 2
    0 2
    0 2
    1 2 
    0 2
    0 2 
    0 2
]

# a bandeira é formada apenas por repetições da primeira coluna, exceto a_8
# que está encaixada na linha 2
C_finland = [
    1 1 1 1 1 1 1 0 1 1 1 1 1 1 1
    0 0 0 0 0 0 0 1 0 0 0 0 0 0 0
]

2×15 Matrix{Int64}:
 1  1  1  1  1  1  1  0  1  1  1  1  1  1  1
 0  0  0  0  0  0  0  1  0  0  0  0  0  0  0

#### Países que possuem bandeira com posto = 3
Noruega e Islândia

Considerando:
- branco = 0
- vermelho = 1
- azul = 2

Com um certo cuidado, podemos perceber que a coluna $a_1$ se repete por quase toda a bandeira. No entanto, na interseção da cruz, temos duas colunas importantes e distintas, que são a cruz central (de única cor) e as linhas laterais. Dessa forma, temos três colunas que não são possíveis de serem formadas a partir das outras, e elas formam o nosso $B$.

In [826]:
A_norway = [
    1 1 1 0 2 2 0 1 1 1 1 1 1 1 1 1
    1 1 1 0 2 2 0 1 1 1 1 1 1 1 1 1
    1 1 1 0 2 2 0 1 1 1 1 1 1 1 1 1
    1 1 1 0 2 2 0 1 1 1 1 1 1 1 1 1
    0 0 0 0 2 2 0 0 0 0 0 0 0 0 0 0
    2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 
    2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
    0 0 0 0 2 2 0 0 0 0 0 0 0 0 0 0
    1 1 1 0 2 2 0 1 1 1 1 1 1 1 1 1
    1 1 1 0 2 2 0 1 1 1 1 1 1 1 1 1
    1 1 1 0 2 2 0 1 1 1 1 1 1 1 1 1
    1 1 1 0 2 2 0 1 1 1 1 1 1 1 1 1
]

12×16 Matrix{Int64}:
 1  1  1  0  2  2  0  1  1  1  1  1  1  1  1  1
 1  1  1  0  2  2  0  1  1  1  1  1  1  1  1  1
 1  1  1  0  2  2  0  1  1  1  1  1  1  1  1  1
 1  1  1  0  2  2  0  1  1  1  1  1  1  1  1  1
 0  0  0  0  2  2  0  0  0  0  0  0  0  0  0  0
 2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2
 2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2
 0  0  0  0  2  2  0  0  0  0  0  0  0  0  0  0
 1  1  1  0  2  2  0  1  1  1  1  1  1  1  1  1
 1  1  1  0  2  2  0  1  1  1  1  1  1  1  1  1
 1  1  1  0  2  2  0  1  1  1  1  1  1  1  1  1
 1  1  1  0  2  2  0  1  1  1  1  1  1  1  1  1

In [827]:
# colunas importantes para a construção do restante da bandeira
B_norway = [
    1 0 2
    1 0 2
    1 0 2
    1 0 2
    0 0 2
    2 2 2
    2 2 2
    0 0 2
    1 0 2
    1 0 2
    1 0 2
    1 0 2
]

12×3 Matrix{Int64}:
 1  0  2
 1  0  2
 1  0  2
 1  0  2
 0  0  2
 2  2  2
 2  2  2
 0  0  2
 1  0  2
 1  0  2
 1  0  2
 1  0  2

In [828]:
# mesma ideia de construção de A a partir das linhas de B
C_norway = [
    1 1 1 0 0 0 0 1 1 1 1 1 1 1 1 1
    0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0
    0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0
]

3×16 Matrix{Int64}:
 1  1  1  0  0  0  0  1  1  1  1  1  1  1  1  1
 0  0  0  1  0  0  1  0  0  0  0  0  0  0  0  0
 0  0  0  0  1  1  0  0  0  0  0  0  0  0  0  0

Analogamente para Islândia:

In [829]:
A_iceland = [
    2 2 2 0 1 1 0 2 2 2 2 2 2 2 2 2
    2 2 2 0 1 1 0 2 2 2 2 2 2 2 2 2
    2 2 2 0 1 1 0 2 2 2 2 2 2 2 2 2
    2 2 2 0 1 1 0 2 2 2 2 2 2 2 2 2
    0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0
    1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
    1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
    0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0
    2 2 2 0 1 1 0 2 2 2 2 2 2 2 2 2
    2 2 2 0 1 1 0 2 2 2 2 2 2 2 2 2
    2 2 2 0 1 1 0 2 2 2 2 2 2 2 2 2
    2 2 2 0 1 1 0 2 2 2 2 2 2 2 2 2
]

# colunas que compõem A e não são possíveis de serem obtidas a partir de combinações de outras
B_iceland = [
    2 0 1
    2 0 1
    2 0 1
    2 0 1
    0 0 1
    1 1 1
    1 1 1
    0 0 1
    2 0 1
    2 0 1
    2 0 1
    2 0 1
]

# construção de A a partir da relação linhaXcoluna com B
C_iceland = [
    1 1 1 0 0 0 0 1 1 1 1 1 1 1 1 1
    0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0
    0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0
]

3×16 Matrix{Int64}:
 1  1  1  0  0  0  0  1  1  1  1  1  1  1  1  1
 0  0  0  1  0  0  1  0  0  0  0  0  0  0  0  0
 0  0  0  0  1  1  0  0  0  0  0  0  0  0  0  0

### Questão 6

Vamos considerar:

- branco = $0$
- cinza = $1$
- preto = $2$

Estamos então aptos a construir uma matriz que represente o desenho:

In [830]:
A = [0 2 0 2 0
     0 2 0 2 0
     0 0 1 0 0
     2 0 0 0 2
     0 2 2 2 0]

5×5 Matrix{Int64}:
 0  2  0  2  0
 0  2  0  2  0
 0  0  1  0  0
 2  0  0  0  2
 0  2  2  2  0

Relembremos que o posto será a quantidade de colunas "independentes", que não podem ser construídas a partir da combinação das outras, da matriz, que nesse caso representa o desenho de um rosto sorrindo.

Pela própria imagem, podemos notar que a primeira coluna é igual à última; a segunda coluna é igual à penúltima e a do parece ser única e não possível de ser formada a partir das outras.

Olhando numericamente para a matriz $A$, comprovaremos essas observações. A coluna $a_5$ é $1a_1$, a coluna $a_4$ é $1a_2$ e a coluna do meio não pode ser formada a partir de combinações das outras.

Sendo assim:

In [831]:
# colunas necessárias para construir a matriz A
B = [0 2 0
     0 2 0
     0 0 1
     2 0 0
     0 2 2]

5×3 Matrix{Int64}:
 0  2  0
 0  2  0
 0  0  1
 2  0  0
 0  2  2

Caso fôssemos reconstruir $A$, precisaríamos multiplicar $B$ por alguma $C$ e retornar às combinações:

In [832]:
# o quanto precisamos (visto pelas colunas) de cada elemento das linhas para reconstruir A
C = [1 0 0 0 1
     0 1 0 1 0
     0 0 1 0 0]

3×5 Matrix{Int64}:
 1  0  0  0  1
 0  1  0  1  0
 0  0  1  0  0

Sendo assim, o posto do sorriso é $3$.

### Questão 9

In [833]:
function find_LU(A)
    """
    A: matriz nxn
    """
    n,n = size(A)
    L = zeros(n,n)
    U = zeros(n,n)

    for i=1:n
        pivot = A[i,i]
        row = A[i,:]
        column = A[:,i]/pivot

        L[:,i] = column #i-ésima coluna da L
        U[i,:] = row #i-ésima linha da U

        A = A - column*row'
    end

    return L,U
end

find_LU (generic function with 1 method)

Declarando a matriz A

In [834]:
A = [2 1 1
     4 5 4
     6 15 16]

3×3 Matrix{Int64}:
 2   1   1
 4   5   4
 6  15  16

A partir do algoritmo na função `find_LU`, encontraremos $L$ e $U$ tal que $A = LU$

In [835]:
L, U = find_LU(A)

([1.0 0.0 0.0; 2.0 1.0 0.0; 3.0 4.0 1.0], [2.0 1.0 1.0; 0.0 3.0 2.0; 0.0 0.0 5.0])

Verificando se temos de fato uma $L$ e uma $U$

In [836]:
L

3×3 Matrix{Float64}:
 1.0  0.0  0.0
 2.0  1.0  0.0
 3.0  4.0  1.0

In [837]:
U

3×3 Matrix{Float64}:
 2.0  1.0  1.0
 0.0  3.0  2.0
 0.0  0.0  5.0

Verificando se $A = LU$, levando em consideração o erro numérico

In [838]:
norm(A - L*U)

0.0

Sabemos que, uma vez que temos $A = LU$, se $Ax = b$, então $LUx = b$. Como temos as funções de substituição reversa e direta, podemos utilizá-las para resolver sistemas lineares desse tipo. 

Primeiro estipularemos $Ux = y$ e resolveremos $Ly = b$. Descobrindo $y$, poderemos resolver $Ux = y$ e encontrar a solução $x$ desejada.

Teremos então três sistemas, sendo eles $LUx = e_i$, $i = 1, 2, 3$

Para $LUx = e_1$

In [839]:
# Ux = y
# Ly = B

b = [1
     0
     0]

# encontrando y tal que Ly = b
y = forward_subs(L, b)
# encontrando x tal que Ux = y
x_c = backward_subs(U, y)

3-element Vector{Float64}:
  0.6666666666666666
 -1.3333333333333333
  1.0

Verificando a corretude

In [840]:
norm(L*U*x_c - b)

0.0

Para $LUx = e_2$

In [841]:
# Ux = y
# Ly = B

b = [0
     1
     0]

# encontrando y tal que Ly = b
y = forward_subs(L, b)
# encontrando x tal que Ux = y
x_d = backward_subs(U, y)

3-element Vector{Float64}:
 -0.033333333333333326
  0.8666666666666667
 -0.8

Verificando a corretude

In [842]:
norm(L*U*x_d - b)

0.0

Para $LUx = e_3$

In [843]:
# Ux = y
# Ly = B

b = [0
     0
     1]

# encontrando y tal que Ly = b
y = forward_subs(L, b)
# encontrando x tal que Ux = y
x_e = backward_subs(U, y)

3-element Vector{Float64}:
 -0.03333333333333334
 -0.13333333333333333
  0.2

Verificando a corretude

In [844]:
norm(L*U*x_e- b)

0.0

Para encontrar a inversa, devemos relembrar que a definição de matriz inversa é: $AA^{-1}=I$, sendo $A$ uma matriz, $A^{-1} a sua inversa e $I$ a matriz identidade.

Consideremos os vetores $x$ dos itens anteriores, sendo eles $x_c, x_d, x_e$, respectivamente em relação a cada questão. Demonstramos que $Ax_c = e_1$, $Ax_d = e_2$, $Ax_e = e_3$. Podemos então colocá-los como colunas de uma matriz e verificar que ela formará a inversa de $A$:

In [845]:
# construindo uma matriz B tal que suas colunas são os vetores x_c, x_d, x_e encontramos anteriormente
B = [x_c x_d x_e]

3×3 Matrix{Float64}:
  0.666667  -0.0333333  -0.0333333
 -1.33333    0.866667   -0.133333
  1.0       -0.8         0.2

Verificando a corretude

In [846]:
#matriz identidade 3x3
I_3x3 = I(3)

norm(A*B - I_3x3)

9.930136612989092e-16

Então a matriz $B$ construída pelos vetores $x$ em suas colunas é a inversa de $A$, ou seja, encontramos $B=A^{-1}$.

Para encontrar a inversa de uma matriz $B \in \mathbb{R}^{n \times n}$, sabendo que temos em mãos $L$ e $U$ tal que $B=LU$, podemos:

lista_de_vetores_x $\leftarrow$ [ ]

B $\leftarrow$ zeros(n, n) // a matriz final, inversa de A

para $i$ em $1...n$:
- y $\leftarrow$ subs_direta($L, e_i$) // resolvendo $Ly = b, b = e_i$
- x $\leftarrow$ subs_reversa($U, y$) // resolvendo $Ux = y$ para encontrar a solução $x$
- B[:, $i$] $\leftarrow$ $x$ // colocando o vetor $x$ encontrando na coluna $i$ da matriz inversa

retorne $B$


Implementando o pseudocódigo para verificar a corretude:

In [847]:
lista_de_vetores_x = []
B = zeros(3, 3)
e = [[1 0 0], [0 1 0], [0 0 1]]'

for i in 1:3
    y = forward_subs(L, e[i])
    x = backward_subs(U, y)
    B[:, i] = x
end

norm(A*B - I(3))

9.930136612989092e-16

Pelas propriedades do determinante, sabemos que, para $A,B \in \mathbb{R}^{n \times n}$, $det(AB) = det(A)det(B)$

No nosso caso, como $A = LU$, temos que $det(A)=det(LU)=det(L)det(U)$. Sabemos calcular a determinante de matrizes triangulates que estejam em $\mathbb{R}^{3 \times 3}$, resultado dado pelo produtório dos elementos da diagonal.

Sendo assim:

In [848]:
det_L = L[1,1]*L[2,2]*L[3,3]

1.0

In [849]:
det_U = U[1,1]*U[2,2]*U[3,3]

30.0

Podemos agora calcular o determinante de A:

In [850]:
det_A = det_L * det_U

30.0

Para encontrar o determinante de uma matriz $B \in \mathbb{R}^{n \times n}$, sabendo sua composição $B=LU$, podemos fazer:

$det_L \leftarrow 1$

$det_U \leftarrow 1$

para $i$ em $1...n$:
- $det_L = det_L \cdot L[i,i]$
- $det_U = det_U \cdot U[i, i]$

$det_A \leftarrow det_L \cdot det_U$ 

retorne $det_A$

Implementando o pseudocódigo para verificar a corretude

In [851]:
det_L = 1
det_U = 1

for i in 1:3
    det_L = det_L * L[i,i]
    det_U = det_U * U[i, i]
end

det_A = det_L * det_U

30.0

#### Questão 10

Modificando a decomposição $LU$ para quando a entrada é uma matriz simétrica (parte teórica feita à mão no PDF).

In [852]:
function find_LU(A)
    """
    A: matriz nxn
    """
    n,n = size(A)
    L = zeros(n,n)
    U = zeros(n,n)

    for i=1:n
        pivot = A[i,i]
        row = A[i,:]
        column = A[:,i]/pivot

        L[:,i] = column #i-ésima coluna da L
        U[i,:] = row #i-ésima linha da U

        A = A - column*row'
    end

    return L,U
end

find_LU (generic function with 1 method)

In [853]:
function find_LU_for_symmetric(A)
    """
    A: matriz nxn simétrica
    """
    n,n = size(A)
    L = zeros(n,n)

    #passo 1 descrito (no PDF)
    L[1,1] = sqrt(A[1,1])

    #passo 2 descrito (calculando a primeira coluna de L)
    for j in 2:n 
        L[j,1] = A[j,1]/L[1,1]
    end

    #passo 3
    for i in 2:n
        # utilizando sum_ como auxiliar para a notação de somatório em matemática
        sum_ = 0
        for k in 1:i-1 # realizando a soma de fato
            sum_ += L[i,k]^2 
        end 
        # calculando a diagonal de L
        L[i,i] = sqrt(A[i,i]-sum_) 
        
        # calculando os elementos abaixo da diagonal
        # passo 4
        for j in i+1:n 
            # novamente utilizando a variável sum_ apenas como suporte para o somatório
            sum_ = 0
            for k in 1:i-1 
                sum_ += L[j,k]*L[i,k]
            end
            # adicionando na matriz L em si
            L[j,i] = (1/L[i,i])*(A[j,i]-sum_)
        end 
    end 

    return L,L'
end

find_LU_for_symmetric (generic function with 1 method)

Utilizando uma matriz qualquer para verificar a decomposição

In [854]:
A = [6 15 55
     15 55 225
     55 225 979]

L, U = find_LU_for_symmetric(A)

([2.449489742783178 0.0 0.0; 6.123724356957946 4.183300132670377 0.0; 22.45365597551247 20.916500663351886 6.110100926607781], [2.449489742783178 6.123724356957946 22.45365597551247; 0.0 4.183300132670377 20.916500663351886; 0.0 0.0 6.110100926607781])

In [855]:
L

3×3 Matrix{Float64}:
  2.44949   0.0     0.0
  6.12372   4.1833  0.0
 22.4537   20.9165  6.1101

In [856]:
U # igual a L'

3×3 adjoint(::Matrix{Float64}) with eltype Float64:
 2.44949  6.12372  22.4537
 0.0      4.1833   20.9165
 0.0      0.0       6.1101

Verificando se $A = LL'$

In [857]:
norm(L*U-A)

8.881784197001252e-16

Reparemos que gastamos menos memória nesse algoritmo pois não precisamos criar uma segunda matriz $U$, basta utilizarmos a própria $L$ transposta como retorno para $U$.

#### Questão 11

Temos que $AZ = LLQU$, $Q$ e $Z$ são ortogonais, $L$ é triangular inferior com diagonal não nula e $U$ é triangular superior com diagonal não nula, sendo todas pertencentes ao $\mathbb{R}^{n \times n}$. 

Com essas informações, queremos determinar $Ax = Lb$ eficientemente, conhecendo de antemão $U$, $L$, $Z$, $Q$ e $b$.

Para alcançar a complexidade $O(n^2)$ na resolução do sistema, será necessário, primeiro, pré-processar $A$, o que terá complexidade $O(n^3)$.

Para isso, precisaremos primeiro notar que:

$AZ = LLQU$

Por $Z$ ser ortogonal, multiplicamos ambos os lados por $Z^t$:

$A = LLQUZ^t$

Temos o sistema $Ax = Lb$, que é o mesmo que $LLQUZ^tx = Lb$. Por $L$ ser triangular inferior com diagonal não nula, ela admite uma inversa; podemos então multiplicar o início de ambas as equações por $L^{-1}$, tornando a $L^{-1}L=I$:

$LQUZ^tx=b$

Agora, computacionalmente, podemos realizar o seguinte procedimento:

Primeiro consideremos $Ly_1=b$, com $y_1=QUZ^tx$
- $y_1 \leftarrow$ subs_direta($L$, $b$)

Teremos ainda o resultado $QUZ^tx = y_1$. Como $Q$ é ortogonal, podemos multiplicar ambos os lados por $Q^t$ e calcularmos numericamente o valor de $Q^ty_1=y_2$:

$UZ^tx = y_2$

Então, considerando $Uy_3=y_2$, com $y_3=Z^tx$:

- $y_3 \leftarrow$ subs_reversa($U$, $y_2$)

Isso nos deixará com

$Z^tx = y_3$

Como $Z^t$ é ortogonal, podemos multiplicar ambos os lados por $Z$ e calcularmos numericamente o valor de $Zy_3=x$:

$x \leftarrow Zy_3$



Sendo assim, conhecendo $U, L, Z, Q$ e $b$, temos que $Ax = Lb$ é o mesmo que $LQUZ^tx=b$. A seguir, o algoritmo:

- $y_1 \leftarrow$ subs_direta($L$,$b$)
- $y_2 \leftarrow Q^ty_1$
- $y_3 \leftarrow$ subs_reversa($U$, $y_2$)
- $x \leftarrow(Z, x)$

retorne $x$

Como todas as operações de subs_direta, subs_reversa e multiplicação matriz x vetor são de complexidade de ordem $O(n^2)$, essa resolução de sistema terá também ordem de $O(n^2)$.

#### Questão 25

Sabemos que temos os vértices $x_1, x_2, x_3$ e $x_4$; e que as temperaturas nas margens são $5, 15, 35$ e $10$ (graus celsius).

Na nossa condição, a temperatura de cada vértice é aproximadamente a média das temperaturas dos $4$ vértices vizinhos.

Sendo assim, temos que:

- $x_1 = \frac{1}{4}(5+15+x_2+x_3)$
- $x_2 = \frac{1}{4}(x_1+15+35+x_3)$
- $x_3 = \frac{1}{4}(x_1+x_4+10+5)$
- $x_4 = \frac{1}{4}(x_2+35+10+x_3)$

Reorganizando as equações:
- $4x_1 = 5+15+x_2+x_3$
- $4x_2 = x_1+15+35+x_3$
- $4x_3 = x_1+x_4+10+5$
- $4x_4 = x_2+35+10+x_3$

Separando as incógnitas:
- $4x_1-x_2-x_3 = 20$
- $4x_2-x_1-x_3 = 50$
- $4x_3-x_1-x_4 = 15$
- $4x_4-x_2-x_3 = 45$

Arrumando a equação de forma a ficar mais legível:
- $4x_1-x_2-x_3+0x_4 = 20$
- $-x_1+4x_2-x_3+0x_4 = 50$
- $-x_1+0x_2+4x_3-x_4 = 15$
- $0x_1-x_2-x_3+4x_4= 45$

Podemos expressar nosso sistema em forma matricial $Ax = b$:

In [858]:
# o lado esquerdo das equações
A = [4 -1 -1 0
    -1  4 -1 0
    -1  0  4 -1
     0 -1 -1 4]

# o lado direito das equações
b = [20
     50
     15
     45]

4-element Vector{Int64}:
 20
 50
 15
 45

Para verificar, podemos utilizar a própria resolução de sistemas lineares do Julia:

In [861]:
x = A\b

4-element Vector{Float64}:
 12.524999999999999
 18.525
 11.575
 18.775

O resultado de fato parece verossímil com as temperaturas das bordas.